In [2]:
library(dplyr)
library(data.table)
library(stats)

In [14]:
infls = snakemake@input$prodigy
infl = infls[[1]]
dfs <- list()
ct <- 0
for (infl in infls) {
    df <- fread(infl,col.names = c("dG","Partner1","Partner2","ID","Run"),sep=" ")
    df$ID <- snakemake@params$id
    ct <- ct + 1
    dfs[[ct]] <- df
}
prodigy <- bind_rows(dfs)

# dfs <- dfs %>%
#     select(dG,Partner1,Partner2,Run,ID)
# fwrite(x = dfs,file = snakemake@output[[1]])
# sumr <- dfs %>%
#     group_by(ID) %>%
#     summarise(dG_median = median(dG,na.rm = T))
# fwrite(x = sumr,file = snakemake@output[[2]])

#snakemake@output
names(prodigy)


In [18]:
infls = snakemake@input$rosetta
infl = infls[[1]]
dfs <- list()
ct <- 0
for (infl in infls) {
    ct <- ct + 1
    df <- fread(infl) %>%
        select(dG_separated,I_sc)
    df$ID <- snakemake@params$id
    df$Run <- ct
    dfs[[ct]] <- df
}
rosetta <- bind_rows(dfs)

In [37]:
data <- rosetta %>%
    left_join(prodigy,by=c("Run","ID")) %>%
    arrange(I_sc) %>%
    slice_head(prop = 0.1)
fwrite(x = dfs,file = snakemake@output[[1]])
sumr <- data %>%
    group_by(ID) %>%
    summarise(
        dG_prodigy = median(dG,na.rm = T),
        dG_separated = median(dG_separated,na.rm = T)
    )
fwrite(x = sumr,file = snakemake@output[[2]])